In [504]:
#importing necessary libraries
import pandas as pd
import numpy as np
import itertools #itertools are functions  creating iterators for efficient looping

In [ ]:
# Himanshu, whats, going,    ---> 0.25
# test --> 0.25 ,  Real

In [485]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [486]:
#importing and reading the dataset
df = pd.read_csv("/content/drive/MyDrive/Github/News Fake Detection/DataSet/news.csv")

df.shape
df.head()

Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL

In [487]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [488]:
df.shape

(6335, 4)

In [489]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [490]:
df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [491]:
df[df.duplicated()].sum()
#there are No dupicates in the data

Unnamed: 0    0.0
title         0.0
text          0.0
label         0.0
dtype: float64

In [492]:
df['Unnamed: 0'].value_counts().sum()

6335

In [493]:
df['title'].value_counts().sum()

6335

In [494]:
df.drop_duplicates(subset='title', keep='first', inplace=True)

In [495]:
df['title'].value_counts().sum()

6256

In [496]:
df['label'].value_counts().sum()

6256

In [497]:
df['Unnamed: 0'].value_counts().sum()

6256

In [498]:
unique_titles = df['title'].unique()
unique_titles_count = len(unique_titles)
print("Number of unique titles:", unique_titles_count)

Number of unique titles: 6256


In [499]:
len(df['Unnamed: 0'].unique())

6256

In [500]:
for i in df.columns:
  print({i:df[i].unique()})

{'Unnamed: 0': array([ 8476, 10294,  3608, ...,  8622,  4021,  4330])}
{'title': array(['You Can Smell Hillary’s Fear',
       'Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)',
       'Kerry to go to Paris in gesture of sympathy', ...,
       'Anti-Trump Protesters Are Tools of the Oligarchy     : Information',
       'In Ethiopia, Obama seeks progress on peace, security in East Africa',
       "Jeb Bush Is Suddenly Attacking Trump. Here's Why That Matters"],
      dtype=object)}
{'text': array(['Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people 

In [501]:
#getting the labels
# labels = df.label
# labels.head()

In [502]:
X = df['text'] #---> text
Y = df['label'] #---> target

In [503]:
#splitting the dataset
from sklearn.model_selection import train_test_split #importing essential library for splitting the dataframe

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 19)

In [478]:
#initialising tfidfvectoriser with stop words from english language and maximum document frequency of 0.7(terms with
# higher frequency would be rejected)
#stop words are most common words in a language that have to be filtered out before processing the natural lang. data
#TFIDF-Vectoriser turns a collection of raw documents into matrix of TF-IDF features.

from sklearn.feature_extraction.text import TfidfVectorizer #fetching vectoriser from sklearn module
# Himanshu,  good wait himanshu  bad --> 0.34
#initializing fit and transform on train set
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df = 0.6) # -- threshold 5 / 6 / 7 / 8

tfidf_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)

In [479]:
# initialising PassiveAggressiveClassifier and fitting it on tfidf_train and y_train
from sklearn.linear_model import PassiveAggressiveClassifier #PassiveAggressiveClassifier typically works with a single target column for binary classification.
from sklearn.metrics import accuracy_score, confusion_matrix

pac=PassiveAggressiveClassifier(max_iter=100)
pac.fit(tfidf_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [480]:
#predicting on test set and calculating the accuracy.
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 94.49%


In [481]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Assuming you have y_test and y_pred from the previous code
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(accuracy*100, 2)}%')
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
# Classification Report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 94.49%
Confusion Matrix:
[[604  35]
 [ 34 579]]
Classification Report:
              precision    recall  f1-score   support

        FAKE       0.95      0.95      0.95       639
        REAL       0.94      0.94      0.94       613

    accuracy                           0.94      1252
   macro avg       0.94      0.94      0.94      1252
weighted avg       0.94      0.94      0.94      1252



In [482]:
#building Confusion Matrix
confusion_matrix(y_test,y_pred) #[590,  35],
                                # [48, 579]

array([[604,  35],
       [ 34, 579]])

In [483]:
#DataFlair - Build confusion matrix
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[604,  35],
       [ 34, 579]])